In [6]:
# -*- coding: utf-8 -*-
from hanspell import spell_checker
from BoxScoreParser import BoxScoreParser
from UrlParser import UrlParser
from sklearn import tree
from sklearn.externals.six import StringIO

from bs4 import BeautifulSoup

import pandas as pd
import numpy as np
import MySQLdb
import random
import urllib2

In [13]:
# DecisionTree - 서두
path_INTRO = './DecisionLearning//DecisionTree_sample.csv'
df_INTRO = pd.read_csv(path_INTRO,index_col=None,header=None,names=['1H','2H','3H','4H','5H','6H','7H','8H','9H','10H','11H','12H','sum','context'])
score_INTRO = np.array(df_INTRO.reindex(columns=['1H','2H','3H','4H','5H','6H','7H','8H','9H','10H','11H','12H','sum']))
context_INTRO = np.array(df_INTRO.get('context'))
context_sen_INTRO = [' ','승리하였다.','접전끝에 승리를 하였다.','역전에 성공했다.','완승을 하였다.','무승부로 끝이났다.','영봉승을 하였다.']
clf_INTRO = tree.DecisionTreeClassifier(criterion='entropy')
clf_INTRO = clf_INTRO.fit(score_INTRO,context_INTRO)

# DecisionTree - 투수
path_MVP_PITCHER = './DecisionLearning/MVP_Pitcher_sample.csv'
df_MVP_PITCHER = pd.read_csv(path_MVP_PITCHER,index_col=None,header=None,names=['ERA','BN','H','SO','context'])
score_MVP_PITCHER = np.array(df_MVP_PITCHER.reindex(columns=['ERA','BN','H','SO']))
context_MVP_PITCHER = np.array(df_MVP_PITCHER.get('context'))
clf_MVP_PITCHER = tree.DecisionTreeClassifier(criterion='entropy')
clf_MVP_PITCHER = clf_MVP_PITCHER.fit(score_MVP_PITCHER,context_MVP_PITCHER)

In [ ]:
def getHighlightInning(url):
    date

In [46]:
data = urllib2.urlopen('http://m.sports.media.daum.net/m/sports/pack/3min/71041490?82573')

In [47]:
data

<addinfourl at 169436872L whose fp = <socket._fileobject object at 0x000000000A17E570>>

In [48]:
html =BeautifulSoup(data)

In [49]:
print html

 <!DOCTYPE html>

<html lang="ko">
<head>
<meta charset="utf-8">
<title>3분 야구</title>
<meta content="IE=edge" http-equiv="X-UA-Compatible">
<meta content="user-scalable=no, initial-scale=1.0, maximum-scale=1.0, minimum-scale=1.0, width=device-width" name="viewport">
<meta content="스포츠뉴스,다음스포츠,스포츠" name="keywords"/>
<meta content="m.sports.media.daum.net" name="svcdomain"/>
<meta content="다음스포츠" property="og:author"/>
<meta content="3일 내내 끝내기로 끝냈다" property="og:title"/>
<meta content="4시간짜리 경기를 3분만에?! 야구 좀 아는 에디터가 엑기스만 뽑아 제대로 정리해 드립니다. 고화질 영상과 뉴스, 경기평점까지 한눈에 보는 고품격 리뷰 'Daum 3분야구'를 만나보세요." property="og:description"/>
<meta content="다음스포츠" property="og:site_name"/>
<meta content="http://t1.daumcdn.net/section/oc/a9ba7df0f6a24fb2ba27295375f71ee7" property="og:image"/>
<meta content="http://m.sports.media.daum.net/m/sports/pack/3min/71041490?82573" property="og:url"/>
<link href="http://i1.daumcdn.net/img-media/2010ci/Daum_favicon.ico" rel="shortcut icon">
<script src="http://s1.daumcdn.net

In [45]:
print html.select('strong')

[]


In [14]:
def writeDate(url):    
    date=bxsParser.date[5:7]+u"월"+bxsParser.date[8:10]+u"일"+bxsParser.date[10:]
    return date

In [15]:
def writeHead(url,contextClassifier,bxsParser):

    # HEAD
    HEAD=u'at ascore vs ht hscore'
    HEAD=HEAD.replace('at',bxsParser.a_team)
    HEAD=HEAD.replace('ht',bxsParser.h_team)
    HEAD=HEAD.replace('hscore',str(bxsParser.h_score[12]))
    HEAD=HEAD.replace('ascore',str(bxsParser.a_score[12]))
    
    return HEAD

In [16]:
def getClassifier(url):
    return clf_INTRO.predict([x-y for x,y in zip(bxsParser.h_score[:13],bxsParser.a_score[:13])])[0]

In [17]:
def writeIntro(url,contextClassifier,bxsParser):
    MVP_PITCHER=None
    
    sentences1=(
        u'winTeam는 date stadium 경기장에서 열린 loseTeam와의 경기에서 ascore-hscore 으로 context',
        u'date stadium에서 열린 at와 ht와의 경기에서 ascore-hscore로 winTeam이 context',
        u'at와 ht의 경기가 date stadium에서 있었던 오늘 at는 ascore, ht는 hscore로 winTeam가 경기에서 context',
        u'date stadium에서 있었던 at와 ht와의 경기, winTeam이 winScore loseTeam이 loseScore로 winTeam이 context',
    )
    sentences2=(
        u'이날 경기에서 mvp_pitcher가 투구수NP 피안타H 의 기록을 보여주며 호투를 하였다.',
        u'이번 경기에서 winTeam의  mvp_pitcher가 INN이닝을 호투하며, R실점 SO삼진을 잡아내며 승리를 이끌었다.'
    )
    
    if bxsParser.a_score[12] > bxsParser.h_score[12]:
        winTeam = bxsParser.a_team
        winScore = bxsParser.a_score[12]
        winTeam_pitRecord = bxsParser.a_pitRecord
        winTeam_batRecord = bxsParser.a_batRecord        
        loseTeam = bxsParser.h_team        
        loseScore = bxsParser.h_score[12]
        loseTeam_pitRecord = bxsParser.h_pitRecord
        loseTeam_batRecord = bxsParser.h_batRecord
        
    else : 
        loseTeam= bxsParser.a_team
        loseScore= bxsParser.a_score[12]
        loseTeam_pitRecord = bxsParser.a_pitRecord
        loseTeam_batRecord= bxsParser.a_batRecord        
        winTeam = bxsParser.h_team        
        winScore = bxsParser.h_score[12]
        winTeam_pitRecord= bxsParser.h_pitRecord
        winTeam_batRecord = bxsParser.h_batRecord
    
    # INTRO1
    INTRO1=sentences1[random.randint(0,len(sentences1)-1)]
    INTRO1=INTRO1.replace('winTeam',winTeam)
    INTRO1=INTRO1.replace('loseTeam',loseTeam)
    INTRO1=INTRO1.replace('winScore',str(winScore))
    INTRO1=INTRO1.replace('loseScore',str(loseScore))
    INTRO1=INTRO1.replace('stadium',bxsParser.stadium)
    INTRO1=INTRO1.replace('hscore',str(bxsParser.h_score[12]))
    INTRO1=INTRO1.replace('ascore',str(bxsParser.a_score[12]))
    INTRO1=INTRO1.replace('date',bxsParser.date)
    INTRO1=INTRO1.replace('ht',bxsParser.h_team)
    INTRO1=INTRO1.replace('at',bxsParser.a_team)    
    
    # INTRO-context
    #'context' -> exception_case(무승부)
    if bxsParser.h_score[12]==bxsParser.a_score[12]:
        INTRO1=INTRO1.replace('context',context_sen_INTRO[5].decode('utf-8'))    
    #'context' -> exception_case(영봉승)
    elif bxsParser.h_score[12]==0 or bxsParser.a_score[12]==0:
        INTRO1=INTRO1.replace('context',context_sen_INTRO[6].decode('utf-8'))    
    #'context'-> 나머지의 경우
    else:
        INTRO1=INTRO1.replace('context',context_sen_INTRO[getClassifier(url)].decode('utf-8'))
        
#     #HOME팀이 이긴 경우 승리->패배
#     if bxsParser.a_score[12] < bxsParser.h_score[12]:
#         INTRO1=INTRO1.replace(u'승리',u'패배')
#         INTRO1=INTRO1.replace(u'성공',u'실패')
#         INTRO1=INTRO1.replace(u'완승',u'완패')
#         INTRO1=INTRO1.replace(u'영봉승',u'영봉패')

#     이날 경기를 승리로 이끈 MVP_PITCHER는 +
#     date.encode('utf8')
#     head.encode('utf8')
#     pre.encode('utf8')
#     url.encode('utf8')
    
    # MVP_PITCHER
    for row in range(len(winTeam_pitRecord)):
        ERA=winTeam_pitRecord[u'평균자책점'][row]
        if type(ERA)==float:
            ERA=float(ERA)
        else:
            ERA=0
        BN=int(winTeam_pitRecord[u'타자'][row])
        H=int(winTeam_pitRecord[u'피안타'][row])
        SO=int(winTeam_pitRecord[u'삼진'][row])
        arr=np.array([ERA,BN,H,SO])
                       
        INTRO2=''
        if clf_MVP_PITCHER.predict(arr)[0]:
            MVP_PITCHER=winTeam_pitRecord[u'선수명'][row] 
            #INTRO2
            INTRO2=sentences2[random.randint(0,len(sentences2)-1)]
            INTRO2=INTRO2.replace('winTeam',winTeam)
            INTRO2=INTRO2.replace('mvp_pitcher',winTeam_pitRecord.iloc[row][u'선수명'])
            INTRO2=INTRO2.replace('ERA',winTeam_pitRecord.iloc[row][u'평균자책점'])
            INTRO2=INTRO2.replace('INN',winTeam_pitRecord.iloc[row][u'이닝'])
            INTRO2=INTRO2.replace('BN',winTeam_pitRecord.iloc[row][u'타자'])
            INTRO2=INTRO2.replace('NP',winTeam_pitRecord.iloc[row][u'투구수'])
            INTRO2=INTRO2.replace('SO',winTeam_pitRecord.iloc[row][u'삼진'])            
            INTRO2=INTRO2.replace('ER',winTeam_pitRecord.iloc[row][u'자책'])
            INTRO2=INTRO2.replace('R',winTeam_pitRecord.iloc[row][u'실점'])
            INTRO2=INTRO2.replace('H',winTeam_pitRecord.iloc[row][u'피안타'])
            break
            
    return INTRO1+INTRO2

In [18]:
# 본문작성
def writeMain(url,contextClassifier,bxsParser):
    pass

In [19]:
# 결론작성
def writeConc(url,contextClassifier,bxsParser):
    pass

In [20]:
def getEmblem(bxsParser):
    if bxsParser.a_score > bxsParser.h_score:
        winTeam = bxsParser.a_team
        winScore = bxsParser.a_score
        winTeam_pitRecord = bxsParser.a_pitRecord
        winTeam_batRecord = bxsParser.a_batRecord        
        loseTeam = bxsParser.h_team        
        loseScore = bxsParser.h_score
        loseTeam_pitRecord = bxsParser.h_pitRecord
        loseTeam_batRecord = bxsParser.h_batRecord
        
    else : 
        loseTeam= bxsParser.a_team
        loseScore= bxsParser.a_score
        loseTeam_pitRecord = bxsParser.a_pitRecord
        loseTeam_batRecord= bxsParser.a_batRecord        
        winTeam = bxsParser.h_team        
        winScore = bxsParser.h_score
        winTeam_pitRecord= bxsParser.h_pitRecord
        winTeam_batRecord = bxsParser.h_batRecord
    
    if winTeam=='삼성':
        emblem='SS'
    elif winTeam=='한화':
        emblem='HH'
    elif winTeam=='SK':
        emblem='SK'
    elif winTeam=='KIA':
        emblem='HT'
    elif winTeam=='LG':
        emblem='LG'
    elif winTeam=='두산':
        emblem='OB'
    elif winTeam=='롯데':
        emblem='LT'
    elif winTeam=='넥센':
        emblem='WO'
    elif winTeam=='kt':
        emblem='KT'
    else:
        emblem='NC'
    return emblem

In [22]:
urlParser = UrlParser('20160628')

In [23]:
# MySQL conf
con=MySQLdb.connect(host='218.150.181.131',user='root',passwd='1234',db='link10th',charset='utf8', use_unicode=True)

for url in urlParser.urlList:
    bxsParser=BoxScoreParser(url)
    date=writeDate(url)
    contextClassifier=getClassifier(url)
    Head=writeHead(url,contextClassifier,bxsParser)
    Intro=writeIntro(url,contextClassifier,bxsParser)
#     Main=writeMain(url,contextClassifier,bxsParser)
#     Conc=writeConc(url,contextClassifier,bxsParser)
    emblem=getEmblem(bxsParser)
    
    #교정부분
    Head = spell_checker.check(Head)
    Intro = spell_checker.check(Intro)
#     Main = spell_checker.check(Main)
#     Conc = spell_checker.check(Conc)
    Head = Head.checked
    Intro = Intro.checked
#     Main = Main.checked
#     Conc = Conc.checked
 
    cursor=con.cursor()
    cursor.execute(
    '''
    INSERT INTO Article(date,Head,Intro,url,emblem)
    VALUES (\'%s\',\'%s\',\'%s\',\'%s\',\'%s\')
    '''
    %(date,Head,Intro,url,emblem))

    # MySQL disconnect
    con.commit()
    cursor.close()
con.close()

C:\Users\HyunYong\AppData\Local\Enthought\Canopy\User\lib\site-packages\sklearn\utils\validation.py:386: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and willraise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
C:\Users\HyunYong\AppData\Local\Enthought\Canopy\User\lib\site-packages\sklearn\utils\validation.py:386: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and willraise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
C:\Users\HyunYong\AppData\Local\Enthought\Canopy\User\lib\site-packages\sklearn\utils\validation.py:386: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and willraise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a 